In [1]:
from transformers import AutoTokenizer
from benchmark_workload_gen import ToolBenchDataLoader, LoadDistribution
num_workloads = 100
num_requests = 4096
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
dataloader = ToolBenchDataLoader('/mnt/data/ssd/sglang_multi_model/multi_node/benchmarks/G1_workload_updated.json', num_workloads, num_requests, tokenizer, LoadDistribution.ALL)


/mnt/data/ssd/sglang_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1582

In [2]:
import random
from dataclasses import dataclass
import re 
@dataclass
class TBOracle:
    trace = {}
    tbl = {}
    num_nodes: int
    counter = {}

    def runtime_selector(self, text: str, request_id: str):
        match = re.search(r'You have access of the following tools:\n1.(.+?): ', text)
        if match:
            tool = match.group(1)
            self.counter[tool] = self.counter.get(tool, 0) + 1
            num_nodes = self.num_nodes
            if tool not in self.tbl:
                self.tbl[tool] = random.randint(0, num_nodes - 1)
            return self.tbl[tool]
        else:
            return random.randint(0, self.num_nodes - 1)


In [7]:
dataloader.data['whatsapp_api']

dict_keys(['whatsapp_api', 'admiralbet', 'fast_email_verifier', 'vod_app', 'cnpj_validator', 'review_generator_ai', 'trading_view', 'fear_and_greed_index', 'bbc_good_food_api', 'transaction', 'geocode_forward_and_reverse', 'bidirectional_text_language_translation', 'twitter_v2_v2', 'covid_news_v2', 'clinicalmarkers', 'amazon_data_scraper_v2', 'foreca_weather', 'game_of_thrones', 'binance_smart_chain', 'green_api', 'flight_integration', 'ai_news_global', 'cloud_cast', 'memes', 'exchange_rate', 'navitime_geocoding', 'catbreeddb', 'cricbuzz_cricket', 'filepursuit', 'tank01_fantasy_stats', 'betsapi', 'rest_country_api', 'flowers', 'premier_league_upcoming_matches', 'similar_web', 'random_word_api', 'tripit', 'get_360_business_tool', 'football_score_api', 'rivet_sms', 'twitter', 'handballapi', 'whatsapp_data', 'amazon_data_v2', 'wonderboyapi', 'article_extractor_and_summarizer', 'testing_for_my_use', 'coinranking', 'crypto_news_v2', 'whin', 'kolektifapi', 'signnow', 'streaming_availability'

In [28]:
import numpy as np
oracle = TBOracle(num_nodes=2)
results = []
for item in workload:
    results.append(oracle.runtime_selector(item[0], request_id=""))
print(np.mean(results))

0.5


In [29]:
count_0 = sum(value in [0] for value in oracle.tbl.values())
count_1 = sum(value in [1] for value in oracle.tbl.values())
print(count_0, count_1)
oracle.tbl

50 50


{'world_time': 0,
 'reqres_dont_change': 1,
 'endpoint_monitor_tool': 0,
 'swagger_petstore': 1,
 'realtor': 0,
 'k_pop': 1,
 'freedom_finance': 0,
 'ai_writer': 1,
 'opensea_data_query': 0,
 'sandbox_mktplace_eu_04_orders': 1,
 'global_recession_live': 0,
 'cat_facts': 1,
 'luhn_algorithm': 0,
 'tmail': 1,
 'crypto_news_v2': 0,
 'aspose_email_cloud': 1,
 'twitter_hashtags': 0,
 'wikihow': 1,
 'instagram_v3': 0,
 'body_mass_index_bmi_calculator': 1,
 'demo_project_v12': 0,
 'shopify_store_scraper': 1,
 'google_jobs': 0,
 'basketball_datafeeds_by_rolling_insights': 1,
 'the_fork_the_spoon': 0,
 'typography': 1,
 'forecast_crypto_and_fiat_currency_exchange_rates': 0,
 'shwe_2d_live_api': 1,
 'lista_de_empresas_por_segmento': 0,
 'drug_info_and_price_history': 1,
 'instagram_data_v2': 0,
 'ptl': 1,
 'coupons': 0,
 'website_analyze_and_seo_audit_pro': 1,
 'defi_data': 0,
 'airplanesdb': 1,
 'opt_nc_public_docker_images': 0,
 'matrimony_profiles': 1,
 'ultimate_tennis': 0,
 'quotsy': 1,
 'e

In [30]:
len(dataloader.data.keys())

1582